In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
df = pd.read_csv("crime_all_filled.csv", encoding='utf-8', low_memory=False)

In [2]:
# 중복된 Case Number만 추출
duplicate_cases = df[df.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 1197


In [ ]:
# 병합 전
print("병합 전 중복:", df.duplicated(subset=['Case Number']).sum())

# 병합 후
df_all = pd.concat([df1, df2, df3], ignore_index=True)
print("병합 후 중복:", df_all.duplicated(subset=['Case Number']).sum())

# 완전 중복 확인
print("완전 중복 수:", df_all.duplicated().sum())

In [3]:
# 1. 시기별 파일 불러오기
df1 = pd.read_csv("crime_2001_2015_filled.csv")
df2 = pd.read_csv("crime_2016_2022_filled.csv")
df3 = pd.read_csv("crime_2023_filled.csv")

In [4]:
# 중복된 Case Number만 추출
duplicate_cases = df1[df1.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 1197


In [5]:
# 중복된 Case Number만 추출
duplicate_cases = df2[df2.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 0


In [6]:
# 중복된 Case Number만 추출
duplicate_cases = df3[df3.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 0


In [ ]:
# 파일로 저장 
df_2001_2015.to_csv("crime_2001_2015.csv", index=False)
df_2016_2022.to_csv("crime_2016_2022.csv", index=False)
df_2023.to_csv("crime_2023.csv", index=False)

In [11]:
df1 = pd.read_csv("crime_2001_2015.csv")
df2 = pd.read_csv("crime_2016_2022.csv")
df3 = pd.read_csv("crime_2023.csv")

In [8]:
# 중복된 Case Number만 추출
duplicate_cases = df1[df1.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 0


In [9]:
# 중복된 Case Number만 추출
duplicate_cases = df2[df2.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 0


In [10]:
# 중복된 Case Number만 추출
duplicate_cases = df3[df3.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 0


In [50]:
def load_ward_gdf(csv_path, ward_col='WARD'):
    df = pd.read_csv(csv_path)
    df = df[df[ward_col].notna()]              # NA 제거
    df = df[df[ward_col] != 'OUT']             # 'OUT' 제거
    df[ward_col] = df[ward_col].astype(float)  # float형 변환
    df['geometry'] = df['the_geom'].apply(wkt.loads)
    return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [51]:
df1 = pd.read_csv("crime_2001_2015.csv")
df1['geometry'] = gpd.points_from_xy(df1['Longitude'], df1['Latitude'])
gdf1 = gpd.GeoDataFrame(df1, geometry='geometry', crs='EPSG:4326')
ward1 = load_ward_gdf("Boundaries - Wards (2003-2015).csv")
joined1 = gpd.sjoin(gdf1, ward1[['WARD', 'geometry']], how='left', predicate='within')
joined1['Ward'] = joined1['Ward'].fillna(joined1['WARD'])  
joined1.drop(columns=['WARD', 'index_right'], inplace=True)
joined1.to_csv("crime_2001_2015_filled.csv", index=False)

In [52]:
df1 = pd.read_csv("crime_2001_2015_filled.csv")

In [53]:
# 중복된 Case Number만 추출
duplicate_cases = df1[df1.duplicated(subset='Case Number', keep=False)]

# 중복된 Case Number 개수 출력
num_duplicates = duplicate_cases['Case Number'].nunique()
print("중복된 Case Number 개수:", num_duplicates)

중복된 Case Number 개수: 1197


In [55]:
# 중복된 Case Number 중에서, 각각의 Case Number에 속한 고유한 Ward 수를 구함
ward_counts_per_case = (
    df1[df1.duplicated(subset='Case Number', keep=False)]
    .groupby('Case Number')['Ward']
    .nunique()
)

# 몇 개의 Case Number가 2개 이상의 Ward에 속해 있는지 확인
ward_multiples = ward_counts_per_case.value_counts().sort_index()

ward_multiples

Ward
1    1178
2      19
Name: count, dtype: int64

In [57]:
import pandas as pd
from collections import Counter

# 먼저 중복된 Case Number 중에서 2개 이상의 Ward에 속하는 경우 필터링
ward_counts_per_case = (
    df1[df1.duplicated(subset='Case Number', keep=False)]
    .groupby('Case Number')['Ward']
    .nunique()
)

# 2개 이상의 Ward에 속하는 Case Number만 선택
multi_ward_cases = ward_counts_per_case[ward_counts_per_case > 1].index

# 해당 Case Number에 속한 데이터만 가져오기
multi_ward_rows = df1[df1['Case Number'].isin(multi_ward_cases)]

# 각 Ward 등장 횟수 세기
ward_freq = Counter(multi_ward_rows['Ward'])

# Pandas 시리즈로 정렬해서 보기 좋게 출력
ward_freq_series = pd.Series(ward_freq).sort_values(ascending=False)

print("여러 개 Ward에 속한 중복 Case Number들 중 가장 자주 등장한 Ward 순위:")
print(ward_freq_series)


여러 개 Ward에 속한 중복 Case Number들 중 가장 자주 등장한 Ward 순위:
7.0     19
10.0    19
dtype: int64


In [58]:
import pandas as pd
from collections import Counter

# 데이터 불러오기
df = pd.read_csv("crime_2001_2015_filled.csv")

# 중복 Case 중 Ward 수 1개인 Case Number 추출
ward_counts_per_case = (
    df[df.duplicated(subset='Case Number', keep=False)]
    .groupby('Case Number')['Ward']
    .nunique()
)

single_ward_cases = ward_counts_per_case[ward_counts_per_case == 1].index
single_ward_rows = df[df['Case Number'].isin(single_ward_cases)]

# 해당 Ward 카운트
ward_distribution = single_ward_rows['Ward'].value_counts().sort_values(ascending=False)

print("중복된 Case 중 Ward가 1개인 케이스에서 Ward별 등장 빈도:")
print(ward_distribution)


중복된 Case 중 Ward가 1개인 케이스에서 Ward별 등장 빈도:
Ward
7.0     1228
10.0    1114
8.0       12
9.0        2
Name: count, dtype: int64


In [23]:
df= pd.read_csv("crime_2001_2015.csv")

In [24]:
sample_df = df.sample(n=1000, random_state=42)
sample_df.to_csv("sample_for_chatgpt.csv", index=False)

In [25]:
sample_df1 = df1.sample(n=1000, random_state=42)
sample_df1.to_csv("sample_for_chatgpt2.csv", index=False)

In [29]:
duplicate_cases.to_csv("sample_for_chatgpt2.csv", index=False)

In [30]:
ward1['WARD'].value_counts()

WARD
4.0     1
22.0    1
36.0    1
48.0    1
27.0    1
50.0    1
7.0     1
15.0    1
34.0    1
40.0    1
10.0    1
2.0     1
35.0    1
33.0    1
32.0    1
17.0    1
21.0    1
16.0    1
45.0    1
42.0    1
13.0    1
14.0    1
43.0    1
29.0    1
44.0    1
46.0    1
24.0    1
23.0    1
49.0    1
37.0    1
18.0    1
31.0    1
25.0    1
8.0     1
26.0    1
28.0    1
3.0     1
47.0    1
1.0     1
38.0    1
11.0    1
30.0    1
39.0    1
12.0    1
9.0     1
6.0     1
5.0     1
19.0    1
41.0    1
20.0    1
Name: count, dtype: int64

In [31]:
from shapely.geometry import MultiPolygon

ward7_geom = ward1[ward1['WARD'] == 7]['geometry'].iloc[0]
type(ward7_geom), isinstance(ward7_geom, MultiPolygon)

(shapely.geometry.multipolygon.MultiPolygon, True)

In [33]:
# 중복된 사건 중 하나의 geometry
point = df1['geometry'].iloc[0]

# 해당 point가 ward1의 어떤 geometry들과 겹치는지 확인
intersecting_wards = ward1[ward1.geometry.intersects(point)]


TypeError: (<class 'geopandas.geoseries.GeoSeries'>, <class 'str'>)

In [38]:
# 비교 결과를 테이블로 보여주기 위한 DataFrame 생성
import pandas as pd

# 사건 Case Number와 해당 위치를 기준으로 두 자료에서 겹치는 Ward 수를 비교
case_number = "HX156738"
comparison_df = pd.DataFrame({
    'Case Number': [case_number],
    'Wards (2003–2015)': [matching_wards],
    'Count (2003–2015)': [len(matching_wards)],
    'Wards (2015–2023)': [matching_wards_2015],
    'Count (2015–2023)': [len(matching_wards_2015)],
    '중복 발생 여부': ['Yes' if len(matching_wards) > 1 else 'No']
})

import ace_tools as tools; tools.display_dataframe_to_user(name="Ward 중복 비교 결과", dataframe=comparison_df

SyntaxError: incomplete input (16654328.py, line 15)

In [41]:
# 두 GeoDataFrame 생성

ward1_gdf = load_ward_gdf("Boundaries - Wards (2015-2023).csv")
ward2_gdf = load_ward_gdf("Boundaries - Wards (2003-2015).csv")

# 사건 위치가 포함되는 Ward를 각각의 GeoDataFrame에서 추출
wards_2003_match = ward1_gdf[ward1_gdf.geometry.intersects(point)][['WARD']]
wards_2015_match = ward2_gdf[ward2_gdf.geometry.intersects(point)][['WARD']]

# 비교 결과를 표로 정리
df = pd.DataFrame({
    'Case Number': ['HX156738'],
    '2003–2015 Matching Wards': [list(wards_2003_match['WARD'])],
    '2015–2023 Matching Wards': [list(wards_2015_match['WARD'])],
    '중복 여부 (2003–2015 기준)': ['Yes' if len(wards_2003_match) > 1 else 'No']
})

import ace_tools as tools; tools.display_dataframe_to_user(name="경계 데이터 기반 Ward 비교", dataframe=ward_compare_df)


TypeError: (<class 'geopandas.geoseries.GeoSeries'>, <class 'str'>)

In [40]:
def load_ward_gdf(csv_path, ward_col='WARD'):
    df = pd.read_csv(csv_path)
    df = df[df[ward_col].notna()]              # NA 제거
    df = df[df[ward_col] != 'OUT']             # 'OUT' 제거
    df[ward_col] = df[ward_col].astype(float)  # float형 변환
    df['geometry'] = df['the_geom'].apply(wkt.loads)
    return gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [42]:

ward1_gdf = load_ward_gdf("Boundaries - Wards (2015-2023).csv")
ward2_gdf = load_ward_gdf("Boundaries - Wards (2003-2015).csv")


In [43]:
ward1_gdf

,the_geom,WARD,SHAPE_Leng,SHAPE_Area,geometry
0,MULTIPOLYGON (((-87.69623470134458 41.85755495...,12.0,93073.340838,1.160965e+08,"MULTIPOLYGON (((-87.69623 41.85755, -87.69625 ..."
1,MULTIPOLYGON (((-87.66288923669032 41.79883809...,16.0,97901.323833,1.031436e+08,"MULTIPOLYGON (((-87.66289 41.79884, -87.66289 ..."
2,MULTIPOLYGON (((-87.69817510963803 41.81729440...,15.0,82183.694820,6.558830e+07,"MULTIPOLYGON (((-87.69818 41.81729, -87.69817 ..."
3,MULTIPOLYGON (((-87.65524133440029 41.80883316...,20.0,90105.195619,1.372904e+08,"MULTIPOLYGON (((-87.65524 41.80883, -87.65524 ..."
4,MULTIPOLYGON (((-87.66420403810295 42.02126158...,49.0,38122.692826,4.973346e+07,"MULTIPOLYGON (((-87.6642 42.02126, -87.66419 4..."
5,MULTIPOLYGON (((-87.71361112161641 41.80053568...,23.0,125226.472414,1.429823e+08,"MULTIPOLYGON (((-87.71361 41.80054, -87.7136 4..."
6,MULTIPOLYGON (((-87.80310674705102 41.94000768...,29.0,107529.243573,1.288191e+08,"MULTIPOLYGON (((-87.80311 41.94001, -87.8031 4..."
7,MULTIPOLYGON (((-87.71438187841963 41.82673338...,14.0,90165.797407,1.430110e+08,"MULTIPOLYGON (((-87.71438 41.82673, -87.71437 ..."
8,MULTIPOLYGON (((-87.61821536534679 41.85542351...,3.0,81530.275167,1.233415e+08,"MULTIPOLYGON (((-87.61822 41.85542, -87.61721 ..."
9,MULTIPOLYGON (((-87.6259709739332 41.873357458...,4.0,119468.403755,1.260069e+08,"MULTIPOLYGON (((-87.62597 41.87336, -87.62597 ..."


In [44]:
ward2_gdf

,the_geom,DATA_ADMIN,PERIMETER,WARD,ALDERMAN,CLASS,WARD_PHONE,HALL_PHONE,HALL_OFFIC,ADDRESS,EDIT_DATE1,SHAPE_AREA,SHAPE_LEN,geometry
0,MULTIPOLYGON (((-87.61720985189288 41.84565509...,9.588452e+07,71516.462726,4.0,WILLIAM BURNS,3.0,773-536-8103,312-744-2690,"121 N LASALLE ST, RM 300 OFFICE 10, 60602",4659 S COTTAGE GROVE STE 203,20030527,9.693978e+07,73428.701824,"MULTIPOLYGON (((-87.61721 41.84566, -87.61692 ..."
1,MULTIPOLYGON (((-87.69441592800892 41.95563904...,6.278870e+07,46189.174036,33.0,RICHARD F. MELL,3.0,773-478-8040,312-744-6825,"121 N LASALLE ST, RM 208, 60602",3649 N KEDZIE AV,20020301,6.278870e+07,46189.173373,"MULTIPOLYGON (((-87.69442 41.95564, -87.69443 ..."
2,MULTIPOLYGON (((-87.6642015105961 42.021260262...,4.685230e+07,44816.944176,49.0,JOSEPH A. MOORE,5.0,773-338-5796,312-744-3067,"121 N LASALLE ST, RM 300 OFFICE 24, 60602",7356 N GREENVIEW AV,20030527,4.682849e+07,45091.156684,"MULTIPOLYGON (((-87.6642 42.02126, -87.66419 4..."
3,MULTIPOLYGON (((-87.77086308398384 41.92416619...,8.526330e+07,60411.795941,37.0,EMMA MITTS,2.0,773-745-2894,312-744-8019,"121 N. LASALLE ST, RM 300, 60602",5344 W NORTH AVE,20030527,8.526330e+07,60411.795499,"MULTIPOLYGON (((-87.77086 41.92417, -87.77085 ..."
4,MULTIPOLYGON (((-87.69312171100901 41.76823092...,1.766406e+08,69225.366759,18.0,LONA LANE,5.0,773-471-1991,312-744-6856,"121 N La Salle St, RM 300",8146 S KEDZIE AVE,02-07-07,1.766406e+08,69225.370710,"MULTIPOLYGON (((-87.69312 41.76823, -87.69312 ..."
5,MULTIPOLYGON (((-87.73460742456527 41.94019448...,7.998288e+07,49778.984047,31.0,RAY SUAREZ,3.0,773-486-6488,312-744-6102,"121 N LASALLE ST, RM 203, 60602",4502 W FULLERTON AV,20030527,7.998288e+07,49778.983534,"MULTIPOLYGON (((-87.73461 41.94019, -87.7346 4..."
6,MULTIPOLYGON (((-87.6471266381175 41.874339325...,1.091780e+08,79433.763485,25.0,DANIEL S. SOLIS,4.0,773-523-4100,312-744-6845,"121 N LASALLE ST, RM 293 OFFICE 14, 60602",2439 S OAKLEY BLVD,20030527,1.091780e+08,79433.762729,"MULTIPOLYGON (((-87.64713 41.87434, -87.64711 ..."
7,MULTIPOLYGON (((-87.58363089689084 41.76443055...,1.546323e+08,70619.852733,8.0,MICHELLE HARRIS,4.0,773-874-3300,312-744-3075,"121 N La Salle St, RM 207",8539 S COTTAGE GROVE AVE,02-07-07,1.546405e+08,70620.116743,"MULTIPOLYGON (((-87.58363 41.76443, -87.58363 ..."
8,MULTIPOLYGON (((-87.71205366334488 41.92292331...,6.900937e+07,72164.074788,26.0,ROBERTO MALDONADO,4.0,773-395-0143,312-744-6853,"121 N LASALLE ST, RM 300 OFFICE 13, 60602",2434 W DIVISION ST,20090910,6.900937e+07,72164.075897,"MULTIPOLYGON (((-87.71205 41.92292, -87.71205 ..."
9,MULTIPOLYGON (((-87.72607211345743 41.89509122...,1.181320e+08,95047.578510,28.0,JASON ERVIN,3.0,773-533-0900,312-744-3066,"121 N LASALLE ST, RM 203 OFFICE 14, 60602",259 N PULASKI RD,20030527,1.181320e+08,95047.578594,"MULTIPOLYGON (((-87.72607 41.89509, -87.72606 ..."


In [48]:
# 각 Ward 데이터셋의 WARD 번호를 확인하여 비교
wards_2003_ids = sorted(ward1_gdf['WARD'].dropna().unique())
wards_2015_ids = sorted(ward2_gdf['WARD'].dropna().unique())

# 두 목록의 차이를 분석
wards_only_in_2003 = set(wards_2003_ids) - set(wards_2015_ids)
wards_only_in_2015 = set(wards_2015_ids) - set(wards_2003_ids)
wards_in_both = set(wards_2003_ids).intersection(set(wards_2015_ids))

comparison_result = {
    "총 Ward 수 (2003–2015)": len(wards_2003_ids),
    "총 Ward 수 (2015–2023)": len(wards_2015_ids),
    "양쪽에 모두 존재하는 Ward 수": len(wards_in_both),
    "2003에만 존재하는 Ward": sorted(list(wards_only_in_2003)),
    "2015에만 존재하는 Ward": sorted(list(wards_only_in_2015))
}

comparison_result

{'총 Ward 수 (2003–2015)': 50,
 '총 Ward 수 (2015–2023)': 50,
 '양쪽에 모두 존재하는 Ward 수': 50,
 '2003에만 존재하는 Ward': [],
 '2015에만 존재하는 Ward': []}

In [49]:
# WARD 7번의 geometry 가져오기
ward_id = 7.0

geom_2015 = ward1_gdf[ward1_gdf['WARD'] == ward_id].geometry.values[0]
geom_2003 = ward2_gdf[ward2_gdf['WARD'] == ward_id].geometry.values[0]

# 비교 결과
print("WARD 7 경계 비교")
print("-" * 30)
print("면적 (2015–2023):", geom_2015.area)
print("면적 (2003–2015):", geom_2003.area)
print("면적 차이:", abs(geom_2015.area - geom_2003.area))
print("경계 동일 여부 (.equals()):", geom_2015.equals(geom_2003))


WARD 7 경계 비교
------------------------------
면적 (2015–2023): 0.001247509811142663
면적 (2003–2015): 0.0011488836713711532
면적 차이: 9.862613977150978e-05
경계 동일 여부 (.equals()): False
